#**Final Project - Basics of deep learning**

#**Name and ID:**

Student 1: shahar zanzuri 209295096




#**Import Libraries**


In [ ]:

%%capture
import re
import math
import functools
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.utils import load_img
from keras.preprocessing import image
from sklearn import metrics
from keras import backend as K
from google.colab import drive
from sklearn.utils import shuffle
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras.models import load_model
import os
import json
import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential, load_model


#**Load Dataset**

Chest X-Ray Images (Pneumonia)

In [ ]:

# Your Kaggle credentials
kaggle_credentials = {
    "username": "shaharzanzuri",
    "key": "19c50658cd70c928a235dd725726f8c1"
}
# Create the Kaggle directory if it doesn't exist
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Write the Kaggle credentials to the kaggle.json file
kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")
with open(kaggle_json_path, "w") as kaggle_file:
    json.dump(kaggle_credentials, kaggle_file)

# Set the appropriate permissions
os.chmod(kaggle_json_path, 0o600)

print("Kaggle credentials set successfully!")

Kaggle credentials set successfully!


In [ ]:
!pip install -q kaggle

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

!unzip -q chest-xray-pneumonia.zip

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.28G/2.29G [00:36<00:00, 99.7MB/s]
100% 2.29G/2.29G [00:36<00:00, 66.9MB/s]


#**Loading Weights**

In [ ]:
!pip install gdown
import gdown


json_url = 'https://drive.google.com/uc?id=1i32WYld7dv9JRH-11x-9nix5T6N7G8Uo'
json_output = 'history_a1.json'

gdown.download(json_url, json_output, quiet=False)





h5_url = 'https://drive.google.com/uc?id=1QyLGDhNkw-fQT64GVi4JCp08ga_dnyFu'    # Modified URL to get the file directly, this file is binary
h5_output = 'model_a1.h5'

gdown.download(h5_url, h5_output, quiet=False)





url = 'https://drive.google.com/uc?id=1IYEYgoOvQqNxwL3FlWF4gVEAv2-j08cf'
output = 'history_a2.json'

gdown.download(url, output, quiet=False)




url = 'https://drive.google.com/uc?id=1QcvHU5HS-AvpR0ewhbPt9A1gACD66qKA'  # Modified URL to get the file directly, this file is binary
output = 'model_a2.h5'

gdown.download(url, output, quiet=False)






# Download the JSON file
url_json = 'https://drive.google.com/uc?id=190g5K5ZiDa4Zez7SPTqTRqoFTpNtcrlw'
output_json = 'history_d.json'

gdown.download(url_json, output_json, quiet=False)


# Download the binary file (assuming it's a TensorFlow model in HDF5 format)
url_h5 = 'https://drive.google.com/uc?id=1c-dwW-5OuMl63HLcA2vNTwOC4CXpRYKR'
output_h5 = 'model_d.h5'

gdown.download(url_h5, output_h5, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1i32WYld7dv9JRH-11x-9nix5T6N7G8Uo
To: /content/history_a1.json
100%|██████████| 686/686 [00:00<00:00, 978kB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1QyLGDhNkw-fQT64GVi4JCp08ga_dnyFu
From (redirected): https://drive.google.com/uc?id=1QyLGDhNkw-fQT64GVi4JCp08ga_dnyFu&confirm=t&uuid=2905f0e9-8ad3-4a40-aa08-f5f26bac9983
To: /content/model_a1.h5
100%|██████████| 169M/169M [00:05<00:00, 33.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IYEYgoOvQqNxwL3FlWF4gVEAv2-j08cf
To: /content/history_a2.json
100%|██████████| 1.10k/1.10k [00:00<00:00, 1.65MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1QcvHU5HS-AvpR0ewhbPt9A1gACD66qKA
From (redirected): https://drive.google.com/uc?id=1QcvHU5HS-AvpR0ewhbPt9A1gACD66qKA&confirm=t&uuid=0780518c-9529-47b9-aef0-3dda61f022b1
To: /content/model_a2.h5
100%|██████████| 255M/255M [00:10<00:00, 25.3MB/s]
Downloading...
From: https://drive.google.com/uc?id

'model_d.h5'

#**Question 1**

#**Question 1.a - Classification Healthy / Sick**

In [ ]:


# Load the saved model
saved_model = load_model('/content/model_a1.h5')

# Load the training history
with open('/content/history_a1.json') as json_file:
    loaded_history = json.load(json_file)

# Function to display the file upload widget
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

def classify_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    classes = ['Healthy', 'Sick']

    result = {
        "class": classes[np.argmax(prediction)],
        "confidence": f"{np.max(prediction) * 100:.2f}%"
    }

    return result

# Get the file path from the user
new_image_path = upload_file()

# Classify the uploaded image
result = classify_image(new_image_path, saved_model)
print(f'The image is classified as {result["class"]} with confidence {result["confidence"]}')


IndexError: list index out of range

#**Question 1.b- Classification healthy / viral pneumonia / bacterial pneumonia**

In [ ]:
from keras.preprocessing import image
from keras.models import load_model
import numpy as np
import json

# Load the saved model
saved_model = load_model('/content/model_a2.h5')

# Load the training history
with open('/content/history_a2.json') as json_file:
    loaded_history = json.load(json_file)

# Function to display the file upload widget
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

# Function to classify the uploaded image
def classify_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    classes = ['Bacterial Pneumonia', 'Healthy', 'Viral Pneumonia']
    result = {
        "class": classes[np.argmax(prediction)],
        "confidence": f"{np.max(prediction) * 100:.2f}%"
    }

    return result

# Get the file path from the user
new_image_path = upload_file()

# Classify the uploaded image
result = classify_image(new_image_path, saved_model)
print(f'The image is classified as {result["class"]} with confidence {result["confidence"]}')


KeyboardInterrupt: 

#**Question 2-  K-Nearest Neighbors (KNN)**


#**KNN for 1.a**

In [ ]:
from keras.preprocessing import image
from keras.models import load_model, Model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os
import numpy as np

# Load the saved model
saved_model = load_model('/content/model_a1.h5')

# Extract the embedding layer from the model
embedding_layer_model = Model(inputs=saved_model.input, outputs=saved_model.get_layer('dense_11').output)

# Function to display the file upload widget
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

# Function to extract features from images
def extract_features(directory, model):
    features = []
    labels = []
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0
            feature = model.predict(img_array)
            features.append(feature.flatten())
            labels.append(class_folder)
    return np.array(features), np.array(labels)

# Get the file path from the user
new_image_path = upload_file()

# Extract features from training data
train_dir = '/content/chest_xray/train'
train_features, train_labels = extract_features(train_dir, embedding_layer_model)

# Split the data for KNN training and testing
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

# Evaluate KNN classifier on test data
y_pred = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of KNN classifier: {accuracy * 100:.2f}%')

# Visualize classes using T-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(train_features)

# Plot T-SNE results
plt.figure(figsize=(10, 8))
for label in np.unique(train_labels):
    indices = np.where(train_labels == label)
    plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], label=label, alpha=0.5)
plt.legend()
plt.title('T-SNE Visualization of Classes')
plt.show()

# Function to classify image using KNN
def classify_image_knn(image_path, knn_model, embedding_model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    feature = embedding_model.predict(img_array).flatten()
    prediction = knn_model.predict([feature])

    return prediction[0]

# Convert the label to uppercase to match the labels in the training data
knn_prediction = classify_image_knn(new_image_path, knn_classifier, embedding_layer_model).upper()

# Print the department name instead of a department number
department_names = {'NORMAL': 'Healthy ', 'PNEUMONIA': 'Sick '}
print(f'The image is classified as {department_names[knn_prediction]} using KNN classifier')
print(f'Accuracy of KNN classifier: {accuracy * 100:.2f}%')


#**KNN for 1.b**

In [ ]:
from google.colab import files


# train
# Organize pneumonia data into viral and bacterial subdirectories
pneumonia_dir = '/content/chest_xray/train/PNEUMONIA'
viral_pneumonia_dir = '/content/chest_xray/train/Viral_Pneumonia'
bacterial_pneumonia_dir = '/content/chest_xray/train/Bacterial_Pneumonia'

os.makedirs(viral_pneumonia_dir, exist_ok=True)
os.makedirs(bacterial_pneumonia_dir, exist_ok=True)

for filename in os.listdir(pneumonia_dir):
    if 'virus' in filename:
        os.rename(os.path.join(pneumonia_dir, filename), os.path.join(viral_pneumonia_dir, filename))
    elif 'bacteria' in filename:
        os.rename(os.path.join(pneumonia_dir, filename), os.path.join(bacterial_pneumonia_dir, filename))

# מחיקת התיקייה המקורית PNEUMONIA
shutil.rmtree(pneumonia_dir)


# Load the saved model
saved_model = load_model('/content/model_a2.h5')

# Extract the embedding layer from the model
embedding_layer_model = Model(inputs=saved_model.input, outputs=saved_model.get_layer('dense_5').output)


# Function to extract features from images
def extract_features(directory, model):
    features = []
    labels = []
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0
            feature = model.predict(img_array)
            features.append(feature.flatten())
            labels.append(class_folder)
    return np.array(features), np.array(labels)

# Function to upload an image from user's computer
def upload_image():
    uploaded = files.upload()
    for fn in uploaded.keys():
        return fn

# Example - upload image
uploaded_image_path = upload_image()

# Define data directories
train_dir = '/content/chest_xray/train'
val_dir = '/content/chest_xray/val'
test_dir = '/content/chest_xray/test'

# Extract features from training data
train_features, train_labels = extract_features(train_dir, embedding_layer_model)

# Split the data for KNN training and testing
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)

# Evaluate KNN classifier on test data
y_pred = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of KNN classifier: {accuracy * 100:.2f}%')

# Visualize classes using T-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(train_features)

# Plot T-SNE results
plt.figure(figsize=(10, 8))
for label in np.unique(train_labels):
    indices = np.where(train_labels == label)
    plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], label=label, alpha=0.5)
plt.legend()
plt.title('T-SNE Visualization of Classes')
plt.show()




# Function to classify image using KNN
def classify_image_knn(image_path, knn_classifier, embedding_model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    feature = embedding_model.predict(img_array).flatten()
    prediction = knn_classifier.predict([feature])

    return prediction[0]

# Example 2 - put your image
new_image_path = uploaded_image_path  # Provide the path to your image
knn_prediction = classify_image_knn(new_image_path, knn_classifier, embedding_layer_model)
# Print the department name instead of a department number
department_names = {'Bacterial_Pneumonia': 'Bacterial Pneumonia', 'NORMAL': 'Healthy ', 'Viral_Pneumonia': 'Viral Pneumonia'}
print(f'The image is classified as {department_names[knn_prediction]} using KNN classifier')
print(f'Accuracy of KNN classifier: {accuracy * 100:.2f}%')

#**Question 4- Anomaly Detection**

In [ ]:


# Load the saved model
saved_model = load_model('/content/model_d.h5')

# Load the training history
with open('/content/history_d.json') as json_file:
    loaded_history = json.load(json_file)

# Function to display the file upload widget
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

# Get the file path from the user
image_path = upload_file()

# Load and preprocess the uploaded image for anomaly detection
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # rescale to [0,1]

# Predict and calculate reconstruction errors
anomaly_predictions = saved_model.predict(img_array, verbose=1)
anomaly_errors = np.mean(np.square(img_array - anomaly_predictions))

# Set a threshold for anomaly detection (you may need to adjust this based on your data)
threshold = 0.0254

# Classify image as healthy or sick based on the threshold
anomaly_label = 'healthy' if anomaly_errors < threshold else 'sick'

print("Anomaly Label:", anomaly_label)
print("Anomaly Errors:", anomaly_errors)
